In [ ]:
from google.colab import files
uploaded = files.upload()
!unzip -q titanic.zip

import pandas as pd
df = pd.read_csv("train.csv")
df.head()

Saving titanic.zip to titanic (2).zip
replace gender_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PassengerId        891 non-null    int64  
 1   Survived           891 non-null    int64  
 2   Pclass             891 non-null    int64  
 3   Name               891 non-null    object 
 4   Age                891 non-null    float64
 5   SibSp              891 non-null    int64  
 6   Parch              891 non-null    int64  
 7   Ticket             891 non-null    object 
 8   Fare               891 non-null    float64
 9   FamilySize         891 non-null    float64
 10  Sex_male           891 non-null    bool   
 11  Embarked_Q         891 non-null    bool   
 12  Embarked_S         891 non-null    bool   
 13  Title_Miss         891 non-null    bool   
 14  Title_Mr           891 non-null    bool   
 15  Title_Mrs          891 non-null    bool   
 16  Title_Rare         891 non

In [ ]:
rows_before = df.shape[0]
print("Before removing duplicates:", rows_before)

Before removing duplicates: 891


In [ ]:
duplicates = df.duplicated()
print("Number of duplicate rows:", duplicates.sum())

Number of duplicate rows: 0


In [ ]:
df = df.drop_duplicates()
rows_after = df.shape[0]
print("After removing duplicates:", rows_after)
print("Number of rows removed:", rows_before - rows_after)

After removing duplicates: 891
Number of rows removed: 0


In [ ]:
missing = df.isnull().sum()
print("Missing values in each column:\n", missing)

Missing values in each column:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [ ]:
df['Age'] = df['Age'].fillna(df['Age'].median())
df = df.drop(columns=['Cabin'])
most_common_embarked = df['Embarked'].mode()[0]
df['Embarked'] = df['Embarked'].fillna(most_common_embarked)
print("Missing values after handling:\n", df.isnull().sum())

Missing values after handling:
 PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


In [ ]:
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

In [ ]:
df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

In [ ]:
df['Title'] = df['Title'].replace(
    ['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev',
     'Sir', 'Jonkheer', 'Dona'],
    'Rare'
)

df['Title'] = df['Title'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})


In [ ]:
df = pd.get_dummies(df, columns=['Sex', 'Embarked', 'Title'], drop_first=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['Age', 'Fare', 'FamilySize']] = scaler.fit_transform(df[['Age', 'Fare', 'FamilySize']])

In [ ]:
df.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,FamilySize,Sex_male,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Rare
0,1,0,3,"Braund, Mr. Owen Harris",-0.565736,1,0,A/5 21171,-0.502445,0.059160,True,False,True,False,True,False,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0.663861,1,0,PC 17599,0.786845,0.059160,False,False,False,False,False,True,False
2,3,1,3,"Heikkinen, Miss. Laina",-0.258337,0,0,STON/O2. 3101282,-0.488854,-0.560975,False,False,True,True,False,False,False
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0.433312,1,0,113803,0.420730,0.059160,False,False,True,False,False,True,False
4,5,0,3,"Allen, Mr. William Henry",0.433312,0,0,373450,-0.486337,-0.560975,True,False,True,False,True,False,False


In [ ]:
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

In [ ]:
fare_outliers, fare_low, fare_high = detect_outliers_iqr(df, 'Fare')
print(f"Number of Fare outliers: {len(fare_outliers)}")

Number of Fare outliers: 116


In [ ]:
age_outliers, age_low, age_high = detect_outliers_iqr(df, 'Age')
print(f"Number of Age outliers: {len(age_outliers)}")

Number of Age outliers: 66


In [ ]:
df['Fare'] = df['Fare'].clip(lower=fare_low, upper=fare_high)
df['Age'] = df['Age'].clip(lower=age_low, upper=age_high)

In [ ]:
print("Fare outliers after capping:", len(detect_outliers_iqr(df, 'Fare')[0]))
print("Age outliers after capping:", len(detect_outliers_iqr(df, 'Age')[0]))

Fare outliers after capping: 0
Age outliers after capping: 0


In [ ]:
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
print("Numeric columns:\n", numerical_cols)
df[numerical_cols].describe()

Numeric columns:
 ['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize']


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,FamilySize
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,8.910000e+02
mean,446.000000,0.383838,2.308642,-0.024769,0.523008,0.381594,-0.164247,-2.392400e-17
std,257.353842,0.486592,0.836071,0.927737,1.102743,0.806057,0.412391,1.000562e+00
min,1.000000,0.000000,1.000000,-2.064308,0.000000,0.000000,-0.648422,-5.609748e-01
25%,223.500000,0.000000,2.000000,-0.565736,0.000000,0.000000,-0.489148,-5.609748e-01
50%,446.000000,0.000000,3.000000,-0.104637,0.000000,0.000000,-0.357391,-5.609748e-01
75%,668.500000,1.000000,3.000000,0.433312,1.000000,0.000000,-0.024246,5.915988e-02
max,891.000000,1.000000,3.000000,1.931883,8.000000,6.000000,0.673106,5.640372e+00


In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
scaler_standard = StandardScaler()
df[['Age_scaled', 'Fare_scaled', 'FamilySize_scaled']] = scaler_standard.fit_transform(
    df[['Age', 'Fare', 'FamilySize']]
)

In [ ]:
scaler_minmax = MinMaxScaler()
df[['Pclass_scaled', 'SibSp_scaled', 'Parch_scaled']] = scaler_minmax.fit_transform(
    df[['Pclass', 'SibSp', 'Parch']]
)

In [35]:
df[['Age_scaled', 'Fare_scaled', 'FamilySize_scaled', 'Pclass_scaled']].describe()

,Age_scaled,Fare_scaled,FamilySize_scaled,Pclass_scaled
count,8.910000e+02,8.910000e+02,8.910000e+02,891.000000
mean,-5.980999e-17,-1.216137e-16,-9.170866e-17,0.654321
std,1.000562e+00,1.000562e+00,1.000562e+00,0.418036
min,-2.199638e+00,-1.174727e+00,-5.609748e-01,0.000000
25%,-5.834321e-01,-7.882908e-01,-5.609748e-01,0.500000
50%,-8.613809e-02,-4.686152e-01,-5.609748e-01,1.000000
75%,4.940382e-01,3.396748e-01,5.915988e-02,1.000000
max,2.110244e+00,2.031623e+00,5.640372e+00,1.000000


In [37]:
df = pd.read_csv("train.csv")

In [38]:
categorical_cols = df.select_dtypes(include='object').columns.tolist()
print("Categorical columns:", categorical_cols)

Categorical columns: ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


In [39]:
df['Title'] = df['Name'].str.extract(r',\s*([^\.]+)\.')
df['Title'] = df['Title'].replace(['Mlle', 'Ms'], 'Miss')
df['Title'] = df['Title'].replace(['Mme'], 'Mrs')
df['Title'] = df['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr',
                                   'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

In [40]:
df = pd.get_dummies(df, columns=['Sex', 'Embarked', 'Title'], drop_first=True)

In [41]:
df.head()
print(df.columns.tolist())

['PassengerId', 'Survived', 'Pclass', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Sex_male', 'Embarked_Q', 'Embarked_S', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Rare', 'Title_the Countess']


In [43]:
df['Age'] = df['Age'].fillna(df['Age'].median())

bins = [0, 12, 19, 35, 60, 80]
labels = ['Child', 'Teen', 'Young_Adult', 'Adult', 'Senior']
df['AgeGroup'] = pd.cut(df['Age'], bins=bins, labels=labels)


In [44]:
df = pd.get_dummies(df, columns=['AgeGroup'], drop_first=True)

In [45]:
df[['Age', 'AgeGroup_Teen', 'AgeGroup_Young_Adult', 'AgeGroup_Adult', 'AgeGroup_Senior']].head()

,Age,AgeGroup_Teen,AgeGroup_Young_Adult,AgeGroup_Adult,AgeGroup_Senior
0,22.0,False,True,False,False
1,38.0,False,False,True,False
2,26.0,False,True,False,False
3,35.0,False,True,False,False
4,35.0,False,True,False,False
